### 測試標註資料

In [ ]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt

path = os.getcwd()
path = os.path.join(path, "by慶偉")
path = os.path.join(path, "ic100-2")
annoabs = os.path.join(path, "annotations")
imgabs = os.path.join(path, "textdet_imgs")

data_type = ["train", "test"]
for datatype in data_type:
    annopath = os.path.join(annoabs, datatype)
    imgpath = os.path.join(imgabs, datatype)
    filelist = [filename for filename in os.listdir(imgpath) if filename.find(".jpg") != -1]

    for filename in filelist:
        
        # txtfilename = filename.replace(".jpg", ".txt")
        imgfilename = "3_0.jpg"
        txtfilename = imgfilename.replace(".jpg", ".txt")

        print(os.path.join(imgpath, imgfilename))
        img = Image.open(os.path.join(imgpath, imgfilename))
        img = np.array(img)

        det_df = pd.read_csv(os.path.join(annopath, f"gt_{txtfilename}"), header=None)
        print(imgfilename)
        print(det_df)

        det_df.columns = ["0", "1", "2", "3", "4", "5", "6", "7", "txt"]
        print(det_df.columns)
        print(det_df["0"].iloc[1])
        pts = [[det_df["0"].iloc[1], det_df["1"].iloc[1]],
               [det_df["2"].iloc[1], det_df["3"].iloc[1]],
               [det_df["4"].iloc[1], det_df["5"].iloc[1]],
               [det_df["6"].iloc[1], det_df["7"].iloc[1]]]
        pts = np.array(pts, np.int32)

        cv2.polylines(img, [pts], isClosed=True, color=(255, 0, 0), thickness=3)

        plt.imshow(img)
        plt.show()

        print(pts)

        sys.exit()

In [ ]:
path = "./data/imgs/CLAHE/0.jpg"
img = cv2.imread(path)
plt.imshow(img)
plt.title(img.shape)
plt.show()



path2 = os.path.join(imgpath, imgfilename)
img2 = Image.open(path2)
img2 = np.array(img2)
points = np.array([[480, 630], [1000, 630], [1000, 750], [480, 750]], np.int32)
text = "G2PWlW07"

cv2.polylines(img2, [points], isClosed=True, color=(255, 0, 0), thickness=3)

plt.imshow(img2)
plt.title(img2.shape)
plt.show()

c_img2 = cv2.rotate(img2, cv2.ROTATE_90_COUNTERCLOCKWISE)

center = (img2.shape[1]//2, img2.shape[0]//2)
angle = 90
rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)

rotated_pts = cv2.transform(np.array([points]), rotation_matrix).squeeze()
rotated_pts = np.round(rotated_pts).astype(int)

plt.imshow(c_img2)
plt.title(c_img2.shape)
plt.show()

### 資料增強 (翻轉)

In [ ]:
from PIL import Image
import pandas as pd
import numpy as np
import os
import cv2


def rotate_imgs(srcimg, srcpts):
    # dstimg = cv2.rotate(srcimg, cv2.ROTATE_90_COUNTERCLOCKWISE)

    center = (srcimg.shape[1]//2, srcimg.shape[0]//2)
    angle = 90
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)

    rotated_pts = cv2.transform(np.array([srcpts]), rotation_matrix).squeeze()
    rotated_pts = np.round(rotated_pts).astype(int)

    ptsdf = pd.DataFrame(rotated_pts, columns=['X', 'Y'])

    # # 打印原始DataFrame
    # print("Original DataFrame:")
    # print(ptsdf)

    # 将DataFrame放置在第一行
    # print(ptsdf.values.flatten())
    return ptsdf.values.flatten()

path = os.getcwd()
path = os.path.join(path, "by慶偉")
path = os.path.join(path, "dataset")

dataType = ["imgs", "annotations"]


for anglestr in [0,90,180]:
    filenameList = [filename for filename in os.listdir(os.path.join(path, dataType[0]))
                    if filename.find(f"_{str(anglestr)}.")!=-1]
    # print(filenameList)
    for imgname in filenameList:
        img = Image.open(os.path.join(os.path.join(path, dataType[0]), imgname))
        img = np.array(img)

        txtname = "gt_"+imgname.replace(".jpg", ".txt")
        df = pd.read_csv(os.path.join(os.path.join(path, dataType[1]), txtname), header=None)
        df.columns = ["0","1","2","3","4","5","6","7","label"]
        # print(df)
        # print(len(df))
        newdf = df
        for rdx in range(len(df)):
            
            aa = df.iloc[rdx][:-1].to_numpy()
            pts = np.array([
                [aa[0],aa[1]],
                [aa[2],aa[3]],
                [aa[4],aa[5]],
                [aa[6],aa[7]]
                ])

            label = df.iloc[rdx][-1]
            # print(pts)
            
            newpts = rotate_imgs(img, pts)
            # print(newpts)

            newdf.iloc[rdx, :-1] = newpts
        # print(newdf)

        dstimg = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
        dstimgname = imgname.replace(f"_{str(anglestr)}.jpg", f"_{str(anglestr+90)}.jpg")
        dsttxtname = txtname.replace(f"_{str(anglestr)}.txt", f"_{str(anglestr+90)}.txt")

        print(os.path.join(os.path.join(path, dataType[0]), dstimgname))
        print(os.path.join(os.path.join(path, dataType[1]), dsttxtname))
        
        # plt.subplot(1,2,1)
        # plt.title(f"{imgname} : {img.shape}")
        # plt.imshow(img)
        # plt.subplot(1,2,2)
        # plt.imshow(dstimg)
        # plt.title(f"{dstimgname} : {dstimg.shape}")
        # plt.show()
        

        dstimgpath = os.path.join(os.path.join(path, dataType[0]), dstimgname)
        # os.remove(dstimgpath)

        # 将 BGR 格式的图像转换为 RGB 格式
        # img_rgb = cv2.cvtColor(dstimg, cv2.COLOR_BGR2RGB)

        # 创建 PIL 图像对象
        pil_img = Image.fromarray(dstimg)

        # 保存图像
        pil_img.save(dstimgpath)
        # cv2.imwrite(dstimgpath, dstimg)
        

        newdf.to_csv(os.path.join(os.path.join(path, dataType[1]), dsttxtname),
                    header=None, index=False)
        # sys.exit()




### data split (train/test/val)

In [ ]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import cv2
import shutil

path = os.getcwd()
path = os.path.join(path, "by慶偉")
srcpath = os.path.join(path, "dataset")
srcimgpath = os.path.join(srcpath, "imgs")
srctxtpath = os.path.join(srcpath, "annotations")

dstpath = os.path.join(path, "ic100-2")

imgfiles = [f"{file}" 
            for file in os.listdir(srcimgpath)]
txtfiles = [f"{file}"
            for file in os.listdir(srctxtpath)]
print(len(imgfiles))
print(len(txtfiles))
df = pd.DataFrame({"imgs":imgfiles, 
                   "annotations":txtfiles})
test_df = df.sample(frac=0.2)
df = df.drop(test_df.index)
df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)


try:
    for file in os.listdir(os.path.join(dstpath, f"textdet_imgs/train/")):
        os.remove(os.path.join(dstpath, f"textdet_imgs/train/{file}"))
except:
    pass
try:
    for file in os.listdir(os.path.join(dstpath, f"textdet_imgs/test/")):
        os.remove(os.path.join(dstpath, f"textdet_imgs/test/{file}"))
except:
    pass
try:
    for file in os.listdir(os.path.join(dstpath, f"annotations/train/")):
        os.remove(os.path.join(dstpath, f"annotations/train/{file}"))
except:
    pass
try:
    for file in os.listdir(os.path.join(dstpath, f"annotations/test/")):
        os.remove(os.path.join(dstpath, f"annotations/test/{file}"))
except:
    pass


for idx in range(len(df)):

    

    src = os.path.join(srcimgpath, df["imgs"].iloc[idx])
    dst = os.path.join(dstpath, f"textdet_imgs/train/{df['imgs'].iloc[idx]}")
    shutil.copyfile(src, dst)

    src2 = os.path.join(srctxtpath, df["annotations"].iloc[idx])
    dst2 = os.path.join(dstpath, f"annotations/train/{df['annotations'].iloc[idx]}")
    shutil.copyfile(src2, dst2)


for idx in range(len(test_df)):

    src = os.path.join(srcimgpath, test_df["imgs"].iloc[idx])
    dst = os.path.join(dstpath, f"textdet_imgs/test/{test_df['imgs'].iloc[idx]}")
    shutil.copyfile(src, dst)

    src2 = os.path.join(srctxtpath, df["annotations"].iloc[idx])
    dst2 = os.path.join(dstpath, f"annotations/test/{test_df['annotations'].iloc[idx]}")
    shutil.copyfile(src2, dst2)

df

### training loss curve (from training log)

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import ast
import matplotlib.pyplot as plt
from time import time, sleep


path = os.getcwd()
path = os.path.join(path, "mmocr")
path = os.path.join(path, "tools")
path = os.path.join(path, "data")
path = os.path.join(path, "ic100-2")
path = os.path.join(path, "20240315_121456")
path = os.path.join(path, "vis_data")
jsonfile = [file for file in os.listdir(path) if file.find(".json") != -1 and file.find("scalar") == -1][0]
js_path = os.path.join(path, jsonfile)

file = []
try:
    with open(js_path, 'r', encoding='utf-8') as f:
        content = f.read()
        # print(content)
        lines = content.strip().split('\n')


        dictionaries = []
        for line in lines:
            # print(line)
            line.replace("NaN", "0")
            try:
                # Use ast.literal_eval instead of eval for safer evaluation
                dictionary = ast.literal_eval(line)
                # Convert values to float, replacing '0' with 0
                dictionary = {k: float(v) if v != '0' else 0 for k, v in dictionary.items()}
                dictionaries.append(dictionary)
            except (SyntaxError, ValueError) as e:
                print(f"Error evaluating line: {line}\nError: {e}")

        # dictionaries = [eval(line) for line in lines]
        # print(dictionaries)

        # try:
        #     dictionary = eval(content)
        #     print(dictionary)
        # except Exception as e:
        #     print(f"Error: {e}")
        # js = json.loads(content)
        # file.append(js)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    # Handle the exception or exit the program if necessary

# Now, you can proceed with the normalization if js is defined successfully
df = pd.DataFrame(dictionaries)

# iterations = [entry["iter"] for entry in dictionaries]
iterations = (df["iter"].values[:])
# loss_values = [entry["loss"] for entry in dictionaries]
loss_values = df["loss"].values[:]
# loss_prob_values = [entry["loss_prob"] for entry in dictionaries]
loss_prob_values = df["loss_prob"].values[:]

# Plot the loss values against iterations
plt.plot(iterations, loss_values, marker='o', linestyle='-', color='b', label='Loss')
plt.plot(iterations, loss_prob_values, marker='s', linestyle='-', color='r', label='Loss Prob')

plt.title('Loss Growth Over Iterations')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

plt.close()


df.to_csv("ic_training_log.csv", index=False)
df